In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, train_loop_no_lagrange, test_loop, test_loop_random_start
from double_net import datasets as ds
import time
import double_net.plot_utils as pu
import importlib
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=1000)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=2)
parser.add_argument('--n-items', type=int, default=3)
parser.add_argument('--num-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=128 * 32)
parser.add_argument('--test-batch-size', type=int, default=1000)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=1000)
parser.add_argument('--rho', type=float, default=1)
parser.add_argument('--rho-incr-iter', type=int, default=2)
parser.add_argument('--rho-incr-amount', type=float, default=1.0)
parser.add_argument('--lagr-update-iter', type=int, default=100)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
parser.add_argument('--sinkhorn-rounds', type=int, default=20)


args = parser.parse_args(args=[])
args

Namespace(batch_size=4096, lagr_update_iter=100, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=2, n_items=3, num_epochs=100, num_examples=524288, random_seed=0, rgt_start=0, rho=1, rho_incr_amount=1.0, rho_incr_iter=2, sinkhorn_epsilon=0.05, sinkhorn_rounds=20, test_batch_size=1000, test_iter=5, test_misreport_iter=1000, test_num_examples=1000)

In [4]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, "")

model = DoubleNet(
    args.n_agents, args.n_items, item_ranges, args.sinkhorn_epsilon, args.sinkhorn_rounds, marginal_choice='unit'
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=args.batch_size, shuffle=True)

In [5]:
%%time
mean_rgt, mean_pay, lagr_mults = train_loop(model, train_loader, args, device=device)

  1%|          | 1/100 [00:59<1:38:24, 59.64s/it]

{'regret_max': 0.4474698603153229, 'regret_mean': 0.026606125757098198, 'regret_mults': tensor([[6.2513, 6.0761]], device='cuda:0'), 'payment': 0.7985957264900208}


  2%|▏         | 2/100 [01:59<1:37:31, 59.71s/it]

{'regret_max': 0.16461628675460815, 'regret_mean': 0.012735269032418728, 'regret_mults': tensor([[7.2990, 7.2231]], device='cuda:0'), 'payment': 0.8505147695541382}


  3%|▎         | 3/100 [02:59<1:36:31, 59.71s/it]

...

{'regret_max': 0.004471480846405029, 'regret_mean': 0.00010979448416037485, 'regret_mults': tensor([[156.9793, 149.7927]], device='cuda:0'), 'payment': 0.8648833632469177}


 99%|█████████▉| 99/100 [1:40:28<01:00, 60.99s/it]

{'regret_max': 0.003989219665527344, 'regret_mean': 0.00011421921226428822, 'regret_mults': tensor([[157.6939, 150.7170]], device='cuda:0'), 'payment': 0.8652638792991638}


100%|██████████| 100/100 [1:41:29<00:00, 60.89s/it]

{'regret_max': 0.004089087247848511, 'regret_mean': 0.00012464250903576612, 'regret_mults': tensor([[159.1331, 152.2217]], device='cuda:0'), 'payment': 0.8650714755058289}
CPU times: user 1h 41min 6s, sys: 12.4 s, total: 1h 41min 19s
Wall time: 1h 41min 29s


In [6]:
dir_name = 'unit_2x3_0_1_experiment_data/'

In [7]:
%%time
# model.sinkhorn_rounds = 100
# model.sinkhorn_epsilon = 1e-2
# test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
test_data = torch.load(dir_name + 'test_data').to(device=device)
cpu_test_data = test_data.clone().to(device='cpu')

test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

result = test_loop(model, test_loader, args, device=device)
result

1it [00:17, 17.59s/it]

CPU times: user 17.5 s, sys: 36 ms, total: 17.6 s
Wall time: 17.6 s


{'payment_mean': 0.8508860468864441,
 'regret_mean': 0.0006896626437082887,
 'regret_max': 0.0031672418117523193}

In [17]:
%%time
args.test_num_examples = 1000
args.test_batch_size = 1000
args.test_misreport_iter = 1000
args.misreport_lr = 1e-1
model.sinkhorn_rounds = 20
model.sinkhorn_epsilon = 5e-2

random_starts = [test_data]
for i in range(10):
    random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
result

1it [03:13, 193.96s/it]

CPU times: user 3min 12s, sys: 478 ms, total: 3min 12s
Wall time: 3min 13s


{'payment_mean': 0.8508860468864441,
 'regret_mean': 0.0007247113389894366,
 'regret_max': 0.00316736102104187}

In [9]:
model.save(dir_name)

In [10]:
import cvxpy as cp
import numpy as np
from typing import NamedTuple
from tqdm.notebook import tqdm

In [11]:
class AuctionResult(NamedTuple):
    alloc: np.ndarray
    welfare: np.ndarray
    payment: np.ndarray

In [12]:
def max_welfare_alloc(bids_mat, k, exact_demand=False):
    # returns allocation, util of allocation

    num_agents = bids_mat.shape[0]
    num_items = bids_mat.shape[1]
    alloc_matrix = cp.Variable((num_agents, num_items), nonneg=True)
    item_supply_constraints = cp.sum(alloc_matrix, axis=0) <= 1
    if exact_demand:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) == k
    else:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) <= k
    welfare = cp.sum((cp.multiply(bids_mat, alloc_matrix)))
    problem = cp.Problem(cp.Maximize(welfare), [item_supply_constraints, agent_demand_constraints])
    problem.solve()
    return (alloc_matrix.value, problem.value)

In [13]:
def vcg_auction(bids_mat, k, exact_demand=False):
    main_alloc, max_welfare_total_util = max_welfare_alloc(bids_mat, k, exact_demand=exact_demand)
    payments = np.zeros(bids_mat.shape[0])
    player_utils = (bids_mat * main_alloc).sum(axis=1)
    num_agents = bids_mat.shape[0]
    for i in range(num_agents):
        dropped_bid_mat = np.delete(bids_mat, (i), axis=0)
        dropped_player_utils = np.delete(player_utils, (i), axis=0) # player utils under full auction
        new_alloc, new_total_util = max_welfare_alloc(dropped_bid_mat, k, exact_demand=exact_demand)
        new_agent_utils = (new_alloc*dropped_bid_mat).sum(axis=1) # player utils without agent i's bid
        payments[i] = (new_agent_utils - dropped_player_utils).sum() 
    return AuctionResult(main_alloc, player_utils, payments)

In [14]:
payments = []
for i in range(test_data.shape[0]):
    payments.append(vcg_auction(cpu_test_data[i].cpu().numpy(), 1, exact_demand=False).payment.sum())

In [15]:
np.mean(payments)

0.04831120999135777

In [16]:
import pickle
to_pkl_lst = [mean_rgt, mean_pay, lagr_mults]
for i, fname in enumerate(['mean_rgt', 'mean_pay', 'lagr_mults']):
    with open(dir_name + fname, 'wb') as fp:
        pickle.dump(to_pkl_lst[i], fp)